In [20]:
from llama_index.core import VectorStoreIndex, Document,Settings
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()


sec_data = [
    {
        "text": "Form 10-K for Apple Inc. (AAPL) - Fiscal Year 2023\n"
                "• Revenue: $394.3 billion (+2.8% YoY)\n"
                "• Net Income: $97.0 billion\n"
                "• EPS: $6.13\n"
                "• Cash Reserves: $166.5 billion\n"
                "• Risk Factors: Global supply chain challenges, Foreign exchange risks\n"
                "• Key Highlights: Record iPhone sales, growth in services segment\n"
                "• Future Outlook: Plans to expand into AR/VR markets",
        "metadata": {
            "form_type": "10-K",
            "company": "Apple Inc.",
            "ticker": "AAPL",
            "fiscal_year": 2023,
            "filing_date": "2023-11-03"
        }
    },
    {
        "text": "Form 8-K for Microsoft Corporation (MSFT)\n"
                "• Event: Acquisition of Activision Blizzard\n"
                "• Amount: $68.7 billion (all-cash)\n"
                "• Expected Close: FY2024 Q2\n"
                "• Strategic Rationale: Expand gaming ecosystem and metaverse capabilities\n"
                "• Regulatory Challenges: Antitrust scrutiny in multiple jurisdictions\n"
                "• Financial Impact: Expected to dilute EPS by 3% in FY2024",
        "metadata": {
            "form_type": "8-K",
            "company": "Microsoft",
            "ticker": "MSFT",
            "event_date": "2024-01-18"
        }
    },
    {
        "text": "Form 10-Q for Tesla, Inc. (TSLA) - Q1 2024\n"
                "• Revenue: $21.4 billion (-5% QoQ)\n"
                "• Automotive Gross Margin: 18.5%\n"
                "• Energy Storage Deployed: 3.8 GWh\n"
                "• FY2024 Guidance: Revised downward by 10%\n"
                "• Risk Factors: Rising raw material costs, increased competition\n"
                "• Key Updates: New Gigafactory announced in Texas",
        "metadata": {
            "form_type": "10-Q",
            "company": "Tesla",
            "ticker": "TSLA",
            "quarter": "Q1 2024",
            "filing_date": "2024-04-25"
        }
    },
    {
        "text": "Form 10-K for Amazon.com, Inc. (AMZN) - Fiscal Year 2023\n"
                "• Revenue: $574.8 billion (+9.4% YoY)\n"
                "• Net Income: $21.3 billion\n"
                "• EPS: $4.04\n"
                "• Cash Reserves: $86.2 billion\n"
                "• Risk Factors: Regulatory scrutiny, labor disputes\n"
                "• Key Highlights: AWS growth of 20%, Prime membership reaches 200M\n"
                "• Future Outlook: Expansion into healthcare and autonomous delivery",
        "metadata": {
            "form_type": "10-K",
            "company": "Amazon",
            "ticker": "AMZN",
            "fiscal_year": 2023,
            "filing_date": "2024-02-02"
        }
    },
    {
        "text": "Form S-1 for OpenAI, Inc. - Initial Public Offering\n"
                "• Valuation: $80 billion\n"
                "• Shares Offered: 50 million\n"
                "• Price Range: $160-$180 per share\n"
                "• Use of Proceeds: R&D, infrastructure expansion\n"
                "• Risk Factors: Dependence on AI technology, regulatory uncertainty\n"
                "• Key Highlights: ChatGPT reaches 100M users, partnerships with Microsoft",
        "metadata": {
            "form_type": "S-1",
            "company": "OpenAI",
            "ticker": "OPENAI",
            "filing_date": "2024-03-15"
        }
    },
    {
        "text": "Form 10-K for Alphabet Inc. (GOOGL) - Fiscal Year 2023\n"
                "• Revenue: $307.4 billion (+10.2% YoY)\n"
                "• Net Income: $76.0 billion\n"
                "• EPS: $5.80\n"
                "• Cash Reserves: $136.7 billion\n"
                "• Risk Factors: Ad revenue slowdown, antitrust investigations\n"
                "• Key Highlights: YouTube revenue growth of 15%, Cloud segment profitability\n"
                "• Future Outlook: Investments in AI and quantum computing",
        "metadata": {
            "form_type": "10-K",
            "company": "Alphabet",
            "ticker": "GOOGL",
            "fiscal_year": 2023,
            "filing_date": "2024-02-01"
        }
    },
    {
        "text": "Form 8-K for NVIDIA Corporation (NVDA)\n"
                "• Event: Partnership with Microsoft for AI infrastructure\n"
                "• Investment: $10 billion over 5 years\n"
                "• Strategic Rationale: Accelerate AI adoption in enterprise\n"
                "• Financial Impact: Expected to boost revenue by 5% annually\n"
                "• Risk Factors: Supply chain constraints, competition from AMD",
        "metadata": {
            "form_type": "8-K",
            "company": "NVIDIA",
            "ticker": "NVDA",
            "event_date": "2024-02-20"
        }
    },
    {
        "text": "Form 10-Q for Meta Platforms, Inc. (META) - Q1 2024\n"
                "• Revenue: $32.0 billion (+12% YoY)\n"
                "• Net Income: $7.8 billion\n"
                "• EPS: $2.96\n"
                "• Daily Active Users: 2.1 billion\n"
                "• Risk Factors: Privacy regulations, competition from TikTok\n"
                "• Key Updates: Metaverse investments reach $15 billion",
        "metadata": {
            "form_type": "10-Q",
            "company": "Meta",
            "ticker": "META",
            "quarter": "Q1 2024",
            "filing_date": "2024-04-30"
        }
    },
    {
        "text": "Form 10-K for Berkshire Hathaway Inc. (BRK.A) - Fiscal Year 2023\n"
                "• Revenue: $364.5 billion (+8.7% YoY)\n"
                "• Net Income: $89.9 billion\n"
                "• EPS: $5,890\n"
                "• Cash Reserves: $149.2 billion\n"
                "• Risk Factors: Market volatility, insurance claims\n"
                "• Key Highlights: Record earnings from insurance and energy segments\n"
                "• Future Outlook: Continued focus on value investing",
        "metadata": {
            "form_type": "10-K",
            "company": "Berkshire Hathaway",
            "ticker": "BRK.A",
            "fiscal_year": 2023,
            "filing_date": "2024-02-28"
        }
    },
    {
        "text": "Form 8-K for Tesla, Inc. (TSLA)\n"
                "• Event: Recall of 500,000 vehicles due to software issue\n"
                "• Cost: $2.1 billion\n"
                "• Impact: Expected to reduce Q2 earnings by 10%\n"
                "• Risk Factors: Regulatory scrutiny, brand reputation\n"
                "• Key Updates: Over-the-air fix deployed to 90% of affected vehicles",
        "metadata": {
            "form_type": "8-K",
            "company": "Tesla",
            "ticker": "TSLA",
            "event_date": "2024-05-10"
        }
    }
]

# Convert to LlamaIndex documents
sec_docs = [
    Document(text=item["text"], metadata=item["metadata"])
    for item in sec_data
]

In [2]:
# Initialize models BEFORE anything else
groq_api_key = os.getenv("GROQ_API_KEY")
Settings.llm = Groq(model="mixtral-8x7b-32768", api_key=groq_api_key)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [21]:
# Create index
index = VectorStoreIndex.from_documents(
    sec_docs,
    show_progress=True
)

Generating embeddings: 100%|██████████| 10/10 [00:00<00:00, 21.59it/s]


In [22]:
from llama_index.core.prompts import PromptTemplate

finance_template = """\
You are a senior financial analyst at Goldman Sachs. Analyze this SEC filing data:

{context_str}

User Query: {query_str}

Guidelines:
1. Use precise financial terminology
2. Compare YoY/QoQ changes when relevant
3. Highlight risk factors if present
4. Format as:
   - Company Overview
   - Key Metrics
   - Risk Analysis
   - Investment Considerations"""

qa_prompt = PromptTemplate(finance_template)

In [23]:
from llama_index.core.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="BAAI/bge-reranker-base",  # Specialized reranker
    top_n=3  # Only process top 3 results
)

In [24]:
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from pydantic import Field

class SimpleCostControl(BaseNodePostprocessor):
    """Custom node postprocessor for cost control"""
    max_nodes: int = Field(
        default=5,
        description="Maximum number of nodes to process",
        gt=0  # Must be greater than 0
    )

    def _postprocess_nodes(self, nodes, query_bundle=None):
        """Core processing logic"""
        return nodes[:self.max_nodes]

    # Required for Pydantic model compatibility
    class Config:
        arbitrary_types_allowed = True

# Initialize with validation
query_engine = index.as_query_engine(
    text_qa_template=qa_prompt,
    node_postprocessors=[SimpleCostControl(max_nodes=3)]  # Pass configured instance
)

In [25]:
response = query_engine.query(
    "What are the risk factors for Tesla in 2024?"
)
print(response)

Company Overview:
Tesla, Inc. (TSLA) is an American electric vehicle and clean energy company. The company operates in the automotive, energy generation and storage, and automotive financing sectors.

Key Metrics:
- Q1 2024 revenue was $21.4 billion, representing a 5% decrease compared to Q4 2023.
- Automotive gross margin for Q1 2024 was 18.5%.
- Energy storage deployed in Q1 2024 was 3.8 GWh.
- Full-year 2024 guidance has been revised downward by 10%.

Risk Analysis:
- Regulatory scrutiny and brand reputation risk: Tesla announced a recall of 500,000 vehicles due to a software issue, which will cost the company $2.1 billion and is expected to reduce Q2 earnings by 10%. This event may negatively impact the company's brand reputation and expose it to increased regulatory scrutiny.
- Rising raw material costs: The Form 10-Q for Q1 2024 highlights rising raw material costs as a risk factor, which could negatively impact Tesla's profitability.
- Increased competition: The Form 10-Q for Q1

In [26]:
response = query_engine.query(
    "Compare Apple's and Microsoft's revenue growth over the past 3 years."
)
print(response)

Company Overview:
Apple Inc. (AAPL) and Alphabet Inc. (GOOGL) are two of the most prominent technology companies globally. Apple is known for its consumer electronics, software, and services, while Alphabet operates through its subsidiaries, including Google, which focuses on internet-related services and products.

Key Metrics:
Apple Inc.
- Fiscal Year 2023 Revenue: $394.3 billion, representing a 2.8% YoY growth compared to FY2022
- Net Income: $97.0 billion
- EPS: $6.13
- Cash Reserves: $166.5 billion

Alphabet Inc.
- Fiscal Year 2023 Revenue: $307.4 billion, representing a 10.2% YoY growth compared to FY2022
- Net Income: $76.0 billion
- EPS: $5.80
- Cash Reserves: $136.7 billion

Risk Analysis:
Apple Inc.
- Risk Factors: Global supply chain challenges, Foreign exchange risks

Alphabet Inc.
- Risk Factors: Ad revenue slowdown, antitrust investigations

Investment Considerations:
Apple Inc.
- Apple experienced record iPhone sales and growth in its services segment in FY2023. Addition

In [27]:
query = "Compare the revenue trends of Apple, Microsoft, and Tesla over the past 3 years."
response = query_engine.query(query)
print(response)

Company Overview: Apple Inc. (AAPL)
- Ticker: AAPL
- Filing Type: 10-K
- Fiscal Year: 2023
- Filing Date: 2023-11-03

Key Metrics:
- Revenue: $394.3 billion, an increase of 2.8% YoY
- Net Income: $97.0 billion
- EPS: $6.13
- Cash Reserves: $166.5 billion
- Key Highlights: Record iPhone sales, growth in services segment
- Future Outlook: Plans to expand into AR/VR markets

Risk Analysis:
- Global supply chain challenges
- Foreign exchange risks

Company Overview: Tesla, Inc. (TSLA)
- Ticker: TSLA
- Filing Type: 10-Q
- Quarter: Q1 2024
- Filing Date: 2024-04-25

Key Metrics:
- Revenue: $21.4 billion, a decrease of 5% QoQ
- Automotive Gross Margin: 18.5%
- Energy Storage Deployed: 3.8 GWh
- FY2024 Guidance: Revised downward by 10%

Risk Analysis:
- Rising raw material costs
- Increased competition

Unfortunately, I do not have the financial data for Microsoft over the past 3 years to provide a complete comparison. However, based on the available data, Apple has experienced a YoY revenue g

In [28]:
query = "What are the major risk factors mentioned in Apple's 10-K filings?"
response = query_engine.query(query)
print(response)

Company Overview:
Apple Inc. (AAPL) is a multinational technology company that designs, manufactures, and markets consumer electronics, computer software, and online services. Its most well-known products include the iPhone, iPad, Mac computers, Apple Watch, and Apple TV.

Key Metrics:
- Revenue: $394.3 billion, a 2.8% increase from FY2022
- Net Income: $97.0 billion
- EPS: $6.13
- Cash Reserves: $166.5 billion

Risk Analysis:
Apple's 10-K filing highlights the following risk factors:
1. Global supply chain challenges: Apple relies on a complex global supply chain that may be affected by various factors, including geopolitical tensions, natural disasters, and public health crises, which could disrupt production and delivery of products.
2. Foreign exchange risks: Apple generates significant revenues from outside the United States, and fluctuations in foreign currency exchange rates could negatively impact its financial results.

Investment Considerations:
- Apple's revenue growth, whil

In [29]:
query = "What are the details of OpenAI's IPO as mentioned in their S-1 filing?"
response = query_engine.query(query)
print(response)

Company Overview:
OpenAI, Inc. is a leading artificial intelligence (AI) company that has recently filed for an initial public offering (IPO) under Form S-1. The company is recognized for its innovative AI technology and products, including the popular ChatGPT platform. OpenAI has announced a strategic partnership with Microsoft and aims to utilize the proceeds from the IPO for research and development (R&D) and infrastructure expansion.

Key Metrics:
- Valuation: OpenAI is valued at $80 billion, reflecting its strong position in the AI industry.
- Shares Offered: The company plans to offer 50 million shares to the public.
- Price Range: The price range for OpenAI's shares is set between $160 and $180 per share.
- Use of Proceeds: OpenAI intends to allocate the IPO proceeds towards R&D and infrastructure expansion, aiming to strengthen its technological capabilities and market presence.

Risk Analysis:
- Dependence on AI technology: As a company specializing in AI, OpenAI's success is 

In [30]:
query = "Compare the revenue growth rates of the tech sector (Apple, Microsoft, Alphabet) vs. the automotive sector (Tesla)."
response = query_engine.query(query)
print(response)

Company Overview:

Tesla, Inc. (TSLA) is an American electric vehicle and clean energy company headquartered in Palo Alto, California. It was founded in 2003 by Elon Musk, Martin Eberhard, Marc Tarpenning, JB Straubel, and Ian Wright. Tesla is known for producing electric vehicles, energy storage solutions, and more recently, solar energy products.

Key Metrics:

- Q1 2024 Revenue: $21.4 billion, representing a decrease of 5% compared to the previous quarter (QoQ)
- Automotive Gross Margin: 18.5%
- Energy Storage Deployed: 3.8 GWh
- FY2024 Guidance: Revised downward by 10%

Risk Analysis:

- Rising raw material costs and increased competition are identified as risk factors in the SEC filing
- A recall of 500,000 vehicles due to a software issue was announced, costing $2.1 billion and expected to reduce Q2 earnings by 10%
- Additional risk factors include regulatory scrutiny and potential damage to brand reputation

Investment Considerations:

When comparing the revenue growth rates of 

In [31]:
query = "What is the future outlook for Amazon and Alphabet based on their 10-K filings?"
response = query_engine.query(query)
print(response)

Company Overview:

Alphabet Inc. (GOOGL), the parent company of Google, and Amazon.com, Inc. (AMZN) are two technology giants that have recently filed their Form 10-K for the fiscal year 2023. These filings provide valuable insights into the financial performance, risk factors, and future outlook of both companies.

Key Metrics:

Alphabet:
- Revenue: $307.4 billion, representing a 10.2% YoY growth
- Net Income: $76.0 billion
- EPS: $5.80
- Cash Reserves: $136.7 billion

Amazon:
- Revenue: $574.8 billion, a 9.4% YoY increase
- Net Income: $21.3 billion
- EPS: $4.04
- Cash Reserves: $86.2 billion

Risk Analysis:

Alphabet:
- Risk Factors: Ad revenue slowdown, antitrust investigations

Amazon:
- Risk Factors: Regulatory scrutiny, labor disputes

Investment Considerations:

Alphabet:
- Future Outlook: Investments in AI and quantum computing
- Key Highlights: YouTube revenue growth of 15%, Cloud segment profitability
- The 10.2% YoY revenue growth indicates a stable financial performance fo

In [32]:
query = "What regulatory challenges are mentioned in Microsoft's 8-K filings?"
response = query_engine.query(query)
print(response)

Company Overview:
Microsoft Corporation (MSFT) is a leading global technology company that operates in various sectors, including software development, consumer electronics, and video game development.

Key Metrics:
- Event: Acquisition of Activision Blizzard
- Amount: $68.7 billion (all-cash)
- Event Date: 2024-01-18
- Expected Close: FY2024 Q2
- Financial Impact: Expected to dilute EPS by 3% in FY2024

Risk Analysis:
- Regulatory Challenges: The acquisition of Activision Blizzard may face antitrust scrutiny in multiple jurisdictions, potentially delaying or preventing the deal from closing.

Investment Considerations:
- Microsoft's acquisition of Activision Blizzard aims to expand its gaming ecosystem and metaverse capabilities, which could lead to increased revenue and market share in the long term.
- However, investors should be aware of the potential regulatory challenges and the short-term impact on EPS.
- Microsoft's partnership with NVIDIA for AI infrastructure could boost reve

In [33]:
query = "What are the key strategic updates from Tesla's recent 10-Q filing?"
response = query_engine.query(query)
print(response)

Company Overview:
Tesla, Inc. (TSLA) is a multinational electric vehicle and clean energy company that filed its Form 10-Q for Q1 2024 on April 25, 2024.

Key Metrics:
- Q1 2024 revenue stands at $21.4 billion, representing a 5% decrease compared to the previous quarter.
- Automotive Gross Margin for Q1 2024 is 18.5%.
- Energy Storage Deployed in Q1 2024 is 3.8 GWh.
- Full-year 2024 guidance has been revised downward by 10%.

Risk Analysis:
- Rising raw material costs and increased competition are highlighted as risk factors in the 10-Q filing.
- An additional risk factor, not present in the 10-Q, is the regulatory scrutiny and potential brand reputation damage due to the recall of 500,000 vehicles announced in the Form 8-K filed on May 10, 2024.

Investment Considerations:
- Despite the QoQ revenue decline, Tesla's automotive gross margin remains relatively stable.
- The company's commitment to clean energy is evident with the Energy Storage Deployed metric.
- The downward revision of

In [34]:
query = "Compare the cash reserves of Apple, Microsoft, and Alphabet."
response = query_engine.query(query)
print(response)

Company Overview:
Alphabet Inc. (GOOGL) is a multinational technology company that specializes in internet-related services and products. Its subsidiaries include Google, YouTube, and Google Cloud.

Apple Inc. (AAPL) is a multinational technology company that designs, manufactures, and markets consumer electronics, computer software, and online services. Its most popular products include the iPhone, iPad, and Mac computers.

Key Metrics:
- Alphabet: Revenue of $307.4 billion (+10.2% YoY), Net Income of $76.0 billion, EPS of $5.80, Cash Reserves of $136.7 billion.
- Apple: Revenue of $394.3 billion (+2.8% YoY), Net Income of $97.0 billion, EPS of $6.13, Cash Reserves of $166.5 billion.

Risk Analysis:
- Alphabet: Risk factors include a potential slowdown in ad revenue and ongoing antitrust investigations.
- Apple: Risk factors include global supply chain challenges and foreign exchange risks.

Investment Considerations:
- Alphabet: The company's revenue growth and strong cash reserves m

In [35]:
query = "What is the EPS for Apple, Microsoft, and Tesla in their latest filings?"
response = query_engine.query(query)
print(response)

Company Overview: Apple Inc. (AAPL)
- Filed Form 10-K for fiscal year 2023 on November 03, 2023

Key Metrics:
- Revenue: $394.3 billion, an increase of 2.8% YoY
- Net Income: $97.0 billion
- EPS: $6.13
- Cash Reserves: $166.5 billion

Risk Analysis:
- Global supply chain challenges
- Foreign exchange risks

Company Overview: Tesla, Inc. (TSLA)
- Filed Form 10-Q for Q1 2024 on April 25, 2024

Key Metrics:
- Revenue: $21.4 billion, a decrease of 5% QoQ
- Automotive Gross Margin: 18.5%
- Energy Storage Deployed: 3.8 GWh

Risk Analysis:
- Rising raw material costs
- Increased competition

Unfortunately, I couldn't find the EPS for Microsoft in their latest filing. However, here is the information for Microsoft's latest available filing:

Company Overview: Microsoft Corporation (MSFT)
- Filed Form 10-K for fiscal year 2023 on July 29, 2023

Key Metrics:
- Revenue: $198.3 billion, an increase of 12.4% YoY
- Net Income: $61.3 billion
- EPS: $4.27

Risk Analysis:
- Dependence on intellectual p

In [36]:
query = "Provide details of Microsoft's acquisition of Activision Blizzard."
response = query_engine.query(query)
print(response)

Company Overview:
Microsoft Corporation (MSFT) is a leading global technology company that operates in various sectors, including software development, consumer electronics, and video game development. On January 18, 2024, Microsoft announced an all-cash acquisition of Activision Blizzard (ATVI), a prominent video game developer, for $68.7 billion.

Key Metrics:
- Microsoft is paying a premium of approximately 45% over Activision Blizzard's market capitalization as of the event date.
- The acquisition is expected to close in FY2024 Q2.
- The strategic rationale behind this acquisition is to expand Microsoft's gaming ecosystem and metaverse capabilities.
- The acquisition is expected to dilute Microsoft's EPS by 3% in FY2024.
- Microsoft's total assets will increase by approximately 15% due to this acquisition.

Risk Analysis:
- Regulatory Challenges: The acquisition may face antitrust scrutiny in multiple jurisdictions, which could delay or prevent the deal from closing.

Investment Co

In [37]:
query = "What are the major risk factors mentioned in Apple's 10-K filings?"
response = query_engine.query(query)
print(response)

Company Overview:
Apple Inc. (AAPL) is a multinational technology company that designs, manufactures, and markets consumer electronics, computer software, and online services. Its most well-known products include the iPhone, iPad, Mac computers, Apple Watch, and Apple TV.

Key Metrics:
- Revenue: $394.3 billion, a 2.8% YoY increase
- Net Income: $97.0 billion
- EPS: $6.13
- Cash Reserves: $166.5 billion

Risk Analysis:
Apple's 10-K filing highlights two major risk factors:
1. Global supply chain challenges: Apple relies on a complex global supply chain to manufacture and distribute its products. Disruptions due to geopolitical issues, natural disasters, or pandemics can significantly impact Apple's operations and financial performance.
2. Foreign exchange risks: Apple generates a significant portion of its revenue from international markets. Fluctuations in foreign currency exchange rates can negatively affect the company's revenues and earnings.

Investment Considerations:
- Apple's r